In [6]:
from flask import Flask, request, make_response, jsonify
from flask_cors import CORS, cross_origin
from scipy.sparse import hstack
import pickle
import string
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
import joblib
from elasticsearch import Elasticsearch, helpers
import time
import pandas as pd
from flask_jwt_extended import create_access_token, get_jwt, jwt_required, JWTManager, set_access_cookies
import numpy as np


nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /home/amogus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/amogus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
app = Flask(__name__)
app.es_client = Elasticsearch(
    "https://127.0.0.1:9200",
    basic_auth=(
        "elastic",
        "yHcm1Pyq=jnDL_4gw93i",
    ),
    ca_certs="~/http_ca.crt",
)
app.recipes_df = pd.read_parquet("~/resources/food/recipes.parquet")
app.user_df = pd.read_parquet("~/resources/food/user.parquet")
app.folder_df = pd.read_parquet("~/resources/food/folder.parquet")
app.bookmark_df = pd.read_parquet("~/resources/food/bookmark.parquet")
app.config["JWT_SECRET_KEY"] = "recipeme79"
app.config["JWT_TOKEN_LOCATION"] = ["cookies"]
app.config["JWT_COOKIE_SAMESITE"] = "None"
app.config["JWT_COOKIE_SECURE"] = True
app.config['CORS_HEADERS'] = 'Content-Type'
cors = CORS(app, resources={r"/foo": {"origins": "http://localhost:5173"}})
jwt = JWTManager(app)


recommend_query = {
    "function_score": {
        "query": {"match_all": {}},
        "functions": [
            {
                "script_score": {
                    "script": {
                        "source": "(doc['AggregatedRating'].value * doc['ReviewCount'].value + 4.632013709922984 * 100) / (doc['AggregatedRating'].value + 100)"
                    },
                },
                "weight": 1,
            },
        ],
        "score_mode": "multiply",
    }
}

def create_folder(username: string, folder_name: string):
    app.folder_df = pd.concat([app.folder_df, pd.DataFrame([[username, folder_name]], columns=["username", "folder_name"])], ignore_index=True)
    app.folder_df.to_parquet("~/resources/food/folder.parquet")
    return app.folder_df

def get_folder_username(username: string):
    return app.folder_df[app.folder_df["username"] == username]

def get_recipes_folder(id: int):
    return app.bookmark_df.loc[app.bookmark_df["folder_index"] == id].join(app.recipes_df, on=["RecipeID"])[["rating"] + list(app.recipes_df.columns)]

def bookmark_recipe(folder_index: int, RecipeID: int, rating: float):
    if (not app.bookmark_df.loc[(app.bookmark_df["folder_index"] == folder_index) & (app.bookmark_df["RecipeID"] == RecipeID)].empty):
        app.bookmark_df.loc[(app.bookmark_df["folder_index"] == folder_index) & (app.bookmark_df["RecipeID"] == RecipeID), "rating"] = rating
    else:
        app.bookmark_df = pd.concat([app.bookmark_df, pd.DataFrame([[folder_index, RecipeID, rating]], columns=["folder_index", "RecipeID", "rating"])], ignore_index=True)
    app.bookmark_df.to_parquet("~/resources/food/folder.parquet")
    return app.bookmark_df

@app.route("/user-detail", methods=["GET"])
@jwt_required()
def get_jwt_data():
    claims = get_jwt()
    return make_response(jsonify(dict(claims)), 200)

@app.route("/login", methods=["POST"])
@cross_origin(origin='localhost',headers=['Content-Type'])
def login():
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    try:
        user = app.user_df.reset_index()[(app.user_df["username"] == username) & (app.user_df["password"] == password)].iloc[0].to_dict()
        print(user)
        additional_claims = {"disp": user.get("display_name")}
        access_token = create_access_token(identity=user.get("username"), additional_claims=additional_claims)
        resp = make_response(jsonify({}) ,200, {"Access-Control-Allow-Origin": "*"})
        set_access_cookies(resp, access_token)
        return resp
    except IndexError:
        return make_response(jsonify({"msg": "Bad username or password"}), 401)


@app.route("/register", methods=["POST"])
@cross_origin(origin='localhost',headers=['Content-Type'])
def register():
    username = request.json.get("username", None)
    password = request.json.get("password", None)
    display_name = request.json.get("display_name", None)

    if app.user_df[(app.user_df["username"] == "username")].shape[0] > 0:
        return make_response(jsonify({"msg": "Bad username"}), 401)
    else:
        app.user_df = pd.concat([app.user_df, pd.DataFrame([[username, password, display_name]], columns=["username", "password", "display_name"])], ignore_index=True)
        app.user_df.to_parquet("~/resources/food/user.parquet")
        additional_claims = {"disp": display_name}
        access_token = create_access_token(identity=username, additional_claims=additional_claims)
        resp = make_response(jsonify({}) ,200)
        set_access_cookies(resp, access_token)
        return resp


def get_search_query(query: str):
    return {
        "function_score": {
            "query": {
                "dis_max": {
                    "queries": [
                        {"match": {"Name": query}},
                        {"match": {"RecipeIngredientParts": query}},
                        {"match": {"RecipeInstructions": query}},
                        {"match": {"Keywords": query}},
                    ],
                    "tie_breaker": 0.3,
                }
            },
            "functions": [
                {
                    "script_score": {
                        "script": {
                            "source": "(doc['AggregatedRating'].value * doc['ReviewCount'].value + 4.632013709922984 * 100) / (doc['AggregatedRating'].value + 100)"
                        },
                    },
                    "weight": 1,
                },
                {
                    "script_score": {
                        "script": {"source": "_score"},
                    },
                    "weight": 1,
                },
            ],
            "score_mode": "multiply",
        }
    }


@app.route("/recommended", methods=["GET"])
@jwt_required()
def get_recommended():
    start = time.time()
    response_object = {"status": "success"}
    results = app.es_client.search(
        index="recipes",
        size=6,
        query=recommend_query,
    )
    end = time.time()
    total_hit = results["hits"]["total"]["value"]
    results_df = pd.DataFrame(
        [[hit["_score"], *hit["_source"].values()] for hit in results["hits"]["hits"]],
        columns=["score"] + list(results["hits"]["hits"][0]["_source"].keys()),
    )
    response_object["total_hit"] = total_hit
    response_object["results"] = results_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/search", methods=["GET"])
@jwt_required()
def search():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    query = argList["query"][0]
    results = app.es_client.search(
        index="recipes",
        size=12,
        query=get_search_query(query),
    )
    end = time.time()
    total_hit = results["hits"]["total"]["value"]
    if len(results["hits"]["hits"]) > 0:
        results_df = pd.DataFrame(
            [[hit["_score"], *hit["_source"].values()] for hit in results["hits"]["hits"]],
            columns=["score"] + list(results["hits"]["hits"][0]["_source"].keys()),
        )
    else:
        results_df = pd.DataFrame()
    response_object["total_hit"] = total_hit
    response_object["results"] = results_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/recipes/<int:id>", methods=["GET"])
@jwt_required()
def get_by_id(id: int):
    start = time.time()
    response_object = {"status": "success"}
    result = app.es_client.get(index="recipes", id=id)
    end = time.time()
    result_df = pd.DataFrame(
        [[*result["_source"].values()]],
        columns=list(result["_source"].keys()),
    )
    response_object["results"] = result_df.to_dict("records")
    response_object["elapse"] = end - start
    return make_response(response_object, 200)


@app.route("/suggest", methods=["GET"])
@jwt_required()
def suggest():
    start = time.time()
    response_object = {"status": "success"}
    argList = request.args.to_dict(flat=False)
    query = argList["query"][0]
    suggest_dictionary = {
        "text": query,
        "suggest-1": {"term": {"field": "all_texts"}},
        "suggest-2": {"term": {"field": "Name"}},
        "suggest-3": {"term": {"field": "Description"}},
        "suggest-4": {"term": {"field": "RecipeInstructions"}},

    }

    query_dictionary = {"suggest": suggest_dictionary}
    res = app.es_client.search(
        index='recipes',
        body=query_dictionary)

    p = []
    for term in np.array(list(res["suggest"].values())).T:
        result = {}
        result["text"] = term[0]["text"]
        options = [v["options"] for v in term]
        result["candidates"] = {}
        # df = pd.DataFrame([v["options"] for v in term])
        for option in options:
            candidates = {}
            if len(option) > 0:
                candidates["text"] = option[0]["text"]
                for candidate in option:
                    # print(candidate)
                    if candidate["text"] not in result["candidates"]:
                        result["candidates"][candidate["text"]] = {"score": candidate["score"], "freq": candidate["freq"]}
                    else:
                        result["candidates"][candidate["text"]]["score"] = (result["candidates"][candidate["text"]]["score"] * result["candidates"][candidate["text"]]["freq"] + candidate["score"] * candidate["freq"]) / (result["candidates"][candidate["text"]]["freq"] + candidate["freq"])
                        result["candidates"][candidate["text"]]["freq"] = result["candidates"][candidate["text"]]["freq"] + candidate["freq"]
        p += [result["candidates"]]
    out = [""] * len(query.split())
    for i, pp in enumerate(p):
        if pp:
            df = pd.DataFrame.from_dict(pp,orient="index")
            R = (df["score"] * df["freq"]).sum() / df["freq"].sum()
            W = df["freq"].mean()
            df["bayes_score"] = (df["score"] * df["freq"] + W * R) / (df["freq"] + W)
            out[i] = df.sort_values("bayes_score", ascending=False).head(1).index[0]
        else:
            out[i] = query.split()[i]
    end = time.time()
    response_object["suggest"] = " ".join(out)
    response_object["elapse"] = end - start
    return make_response(response_object, 200)

@app.after_request
def apply_caching(response):
    response.headers["Access-Control-Allow-Origin"] = "http://localhost:5173"
    response.headers["Access-Control-Allow-Headers"] = "Origin, X-Requested-With, Content-Type, Accept"
    response.headers["Access-Control-Allow-Credentials"] = "true"
    return response


In [8]:
app.run(debug=False, host="0.0.0.0", ssl_context='adhoc')


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:5000
 * Running on https://10.148.0.2:5000
Press CTRL+C to quit
180.183.224.101 - - [15/Mar/2024 17:10:49] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:10:54] "GET /search?query=beef HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:10:54] "GET /suggest?query=beef HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:10:58] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:10:58] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:54] "OPTIONS /login HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:54] "POST /login HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:54] "GET /user-detail HTTP/1.1" 200 -


{'index': 0, 'username': 'test', 'password': 'test', 'display_name': 'test-user'}


180.183.224.101 - - [15/Mar/2024 17:11:55] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:56] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:57] "GET /recipes/89204 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:58] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:11:58] "GET /recipes/27208 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:14:25] "OPTIONS /login HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:16:08] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:16:11] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:16:12] "GET /search?query=beef HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:16:12] "GET /suggest?query=beef HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:18:53] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:18:53] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:18:54] "GET /recipes/

{'index': 3, 'username': 'test3', 'password': 'test3', 'display_name': 'Gorgon Ramsey'}


180.183.224.101 - - [15/Mar/2024 17:22:36] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:22:40] "GET /recipes/27208 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:22:41] "GET /recipes/89204 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:22:41] "GET /recipes/2886 HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:22:43] "GET /recommended HTTP/1.1" 200 -
180.183.224.101 - - [15/Mar/2024 17:22:44] "GET /recipes/39087 HTTP/1.1" 200 -
